# Phase 1 -- Cell Tower Rent Intelligence: Analytical Framework

**Purpose:** Pure analytics and visualization layer (no ML) that lets the business
understand what we are paying, how it compares to regional peers, and where
anomalies exist -- decomposed across six factor dimensions.

| Step | Section | Objective |
|------|---------|---------|
| 1 | Data Preparation | Load, validate, build geographic hierarchy, derive contractual metrics |
| 2 | Map Layer | Interactive map of all sites in a submarket, color-coded by rent |
| 3 | Search Ring Engine | Select a center point, draw 5/10/15-mile rings, identify sites |
| 4 | Aggregation Layer | Site vs. submarket and ring comparison (z-score, percentile, deviation) |
| 5 | Factor Comparison | Six panels: Macro, Micro, Zoning, Technology, Competition, Contractual |
| 6 | Business KPIs | 29 KPIs across portfolio, site, and trend levels |
| 7 | Anomaly Detection | Five statistical methods, severity tiering, summary table |
| 8 | Executive View | Integrated dashboard output |

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import stats as sp_stats
import math
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

pd.set_option('display.max_columns', 80)
pd.set_option('display.float_format', '{:,.2f}'.format)

PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
PARQUET_PATH = DATA_DIR / 'synthetic' / 'synthetic_5k_sites.parquet'

MILES_TO_KM = 1.60934
CPI_AVG_ANNUAL = 0.025

print(f'Project root : {PROJECT_ROOT}')
print(f'Data file    : {PARQUET_PATH}')
print(f'File exists  : {PARQUET_PATH.is_file()}')

---
## Step 1 -- Data Preparation & Market Hierarchy

In [ ]:
if PARQUET_PATH.is_file():
    df = pd.read_parquet(PARQUET_PATH)
    print(f'Loaded {len(df):,} sites from parquet')
else:
    print('Parquet not found -- running sim_00 data generator inline ...')
    %run ../simulations/sim_00_data_generator.ipynb
    df = pd.read_parquet(PARQUET_PATH)
    print(f'Generated and loaded {len(df):,} sites')

print(f'Shape: {df.shape}')
print(f'Columns ({len(df.columns)}): {list(df.columns)}')

In [ ]:
# ── 1a. Geographic hierarchy ──────────────────────────────────────────
df['submarket'] = df['metro_region']

state_order = sorted(df['state'].unique())
submarket_order = sorted(df['submarket'].unique())

hierarchy = (
    df.groupby(['state', 'submarket'])
    .agg(site_count=('site_id', 'count'),
         median_rent=('current_monthly_rent', 'median'))
    .reset_index()
    .sort_values(['state', 'submarket'])
)

print(f'States       : {df["state"].nunique()}')
print(f'Submarkets   : {df["submarket"].nunique()}')
print(f'\nHierarchy (State -> Submarket -> # Sites):')
for st in state_order:
    subs = hierarchy[hierarchy['state'] == st]
    names = ', '.join(f"{r['submarket']} ({r['site_count']})" for _, r in subs.iterrows())
    print(f'  {st}: {names}')

In [ ]:
# ── 1b. Validate lat/lon and rent fields ─────────────────────────────
missing_coords = df[['lat', 'lon']].isnull().any(axis=1).sum()
missing_rent = df['current_monthly_rent'].isnull().sum()

print(f'Missing coordinates : {missing_coords}')
print(f'Missing rent        : {missing_rent}')
print(f'Lat  range          : [{df["lat"].min():.2f}, {df["lat"].max():.2f}]')
print(f'Lon  range          : [{df["lon"].min():.2f}, {df["lon"].max():.2f}]')
print(f'Rent range          : [${df["current_monthly_rent"].min():,.0f}, ${df["current_monthly_rent"].max():,.0f}]')
print(f'Rent median         : ${df["current_monthly_rent"].median():,.0f}')

In [ ]:
# ── 1c. Derive contractual metrics ────────────────────────────────────
df['escalation_drift_ratio'] = (
    df['cumulative_escalation_factor']
    / (1 + CPI_AVG_ANNUAL) ** df['lease_age_years']
)

df['rent_vintage_gap'] = df['current_monthly_rent'] - df['de_escalated_base_rent']

df['rent_vintage_gap_pct'] = (
    df['rent_vintage_gap'] / df['current_monthly_rent'] * 100
).clip(0, 100)

df['term_urgency_score'] = 1.0 / df['remaining_term_years'].clip(lower=0.5)

df['amendment_adjusted_flag'] = (df['amendment_count'] > 0).astype(int)

# Technology label mapping
tech_map = {1: '2G', 2: '3G', 3: '4G', 4: '5G_sub6', 5: '5G_mmW'}
df['technology_label'] = df['primary_technology'].map(tech_map)

print('Derived contractual metrics:')
print(df[['site_id', 'escalation_drift_ratio', 'rent_vintage_gap',
          'rent_vintage_gap_pct', 'term_urgency_score',
          'amendment_adjusted_flag']].describe().round(3))

---
## Step 2 -- Interactive Map Layer

In [ ]:
def plot_submarket_map(df, submarket, color_col='current_monthly_rent'):
    """Plot all sites in a submarket on an interactive map."""
    sub = df[df['submarket'] == submarket].copy()
    center_lat, center_lon = sub['lat'].median(), sub['lon'].median()

    fig = px.scatter_mapbox(
        sub,
        lat='lat', lon='lon',
        color=color_col,
        size='tenant_count',
        color_continuous_scale='RdYlGn_r',
        hover_name='site_id',
        hover_data={
            'current_monthly_rent': ':$,.0f',
            'property_type': True,
            'technology_label': True,
            'tenant_count': True,
            'lease_age_years': ':.1f',
            'escalation_type': True,
            'lat': ':.4f', 'lon': ':.4f',
        },
        title=f'{submarket} -- {len(sub)} Cell Sites (color = {color_col})',
        mapbox_style='carto-positron',
        zoom=8,
        center={'lat': center_lat, 'lon': center_lon},
        height=600,
    )
    fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))
    return fig


DEMO_SUBMARKET = 'Dallas'
fig_map = plot_submarket_map(df, DEMO_SUBMARKET)
fig_map.show()

---
## Step 3 -- Search Ring Engine

In [ ]:
def haversine_miles(lat1, lon1, lat2, lon2):
    """Haversine distance in miles between two WGS-84 points (vectorised)."""
    R_MI = 3958.8
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return R_MI * 2 * np.arcsin(np.sqrt(a))


def find_sites_in_rings(df, center_lat, center_lon,
                        radii_miles=(5, 10, 15)):
    """
    Tag every site with its distance from center and ring band.
    Returns a copy with columns: distance_mi, ring_band.
    """
    out = df.copy()
    out['distance_mi'] = haversine_miles(
        center_lat, center_lon,
        out['lat'].values, out['lon'].values
    )
    radii = sorted(radii_miles)
    conditions = [
        out['distance_mi'] <= radii[0],
        out['distance_mi'] <= radii[1],
        out['distance_mi'] <= radii[2],
    ]
    labels = [
        f'0-{radii[0]} mi',
        f'{radii[0]}-{radii[1]} mi',
        f'{radii[1]}-{radii[2]} mi',
    ]
    out['ring_band'] = np.select(conditions, labels, default='Outside')
    return out


def make_ring_polygon(center_lat, center_lon, radius_mi, n_points=72):
    """Generate lat/lon points for a circle overlay on the map."""
    R_MI = 3958.8
    lats, lons = [], []
    for angle in np.linspace(0, 2 * np.pi, n_points):
        d = radius_mi / R_MI
        lat1 = math.radians(center_lat)
        lon1 = math.radians(center_lon)
        lat2 = math.asin(math.sin(lat1) * math.cos(d) +
                         math.cos(lat1) * math.sin(d) * math.cos(angle))
        lon2 = lon1 + math.atan2(
            math.sin(angle) * math.sin(d) * math.cos(lat1),
            math.cos(d) - math.sin(lat1) * math.sin(lat2))
        lats.append(math.degrees(lat2))
        lons.append(math.degrees(lon2))
    lats.append(lats[0])
    lons.append(lons[0])
    return lats, lons


print('Search-ring functions defined.')

In [ ]:
# ── Demo: Pick center of Dallas submarket and draw rings ─────────────
sub_dallas = df[df['submarket'] == DEMO_SUBMARKET]
CENTER_LAT = sub_dallas['lat'].median()
CENTER_LON = sub_dallas['lon'].median()

ring_df = find_sites_in_rings(df, CENTER_LAT, CENTER_LON)
ring_df_in = ring_df[ring_df['ring_band'] != 'Outside'].copy()

print(f'Center: ({CENTER_LAT:.4f}, {CENTER_LON:.4f})')
print(f'\nSites within search rings:')
print(ring_df_in['ring_band'].value_counts().sort_index())

band_order = ['0-5 mi', '5-10 mi', '10-15 mi']
color_map = {'0-5 mi': '#d62728', '5-10 mi': '#ff7f0e', '10-15 mi': '#2ca02c'}

fig_ring = px.scatter_mapbox(
    ring_df_in, lat='lat', lon='lon',
    color='ring_band',
    color_discrete_map=color_map,
    category_orders={'ring_band': band_order},
    hover_name='site_id',
    hover_data={'current_monthly_rent': ':$,.0f', 'distance_mi': ':.1f',
                'property_type': True, 'submarket': True},
    title=f'Search Rings from ({CENTER_LAT:.2f}, {CENTER_LON:.2f})',
    mapbox_style='carto-positron',
    zoom=7, center={'lat': CENTER_LAT, 'lon': CENTER_LON},
    height=600,
)

ring_colors = ['rgba(214,39,40,0.25)', 'rgba(255,127,14,0.2)', 'rgba(44,160,44,0.15)']
for radius, color in zip([5, 10, 15], ring_colors):
    lats, lons = make_ring_polygon(CENTER_LAT, CENTER_LON, radius)
    fig_ring.add_trace(go.Scattermapbox(
        lat=lats, lon=lons, mode='lines',
        fill='toself', fillcolor=color,
        line=dict(width=1.5, color=color.replace('0.25', '0.8').replace('0.2', '0.8').replace('0.15', '0.8')),
        name=f'{radius}-mile ring',
        showlegend=True,
    ))

fig_ring.add_trace(go.Scattermapbox(
    lat=[CENTER_LAT], lon=[CENTER_LON], mode='markers',
    marker=dict(size=14, color='black', symbol='star'),
    name='Center', showlegend=True,
))

fig_ring.update_layout(margin=dict(l=0, r=0, t=40, b=0))
fig_ring.show()

---
## Step 4 -- Aggregation Layer: Site vs. Region Comparison

In [ ]:
# ── 4a. Submarket-level aggregates ────────────────────────────────────
submarket_stats = (
    df.groupby('submarket')['current_monthly_rent']
    .agg(['median', 'mean', 'std',
          lambda x: x.quantile(0.25),
          lambda x: x.quantile(0.75)])
    .rename(columns={'median': 'sub_median', 'mean': 'sub_mean',
                     'std': 'sub_std',
                     '<lambda_0>': 'sub_p25', '<lambda_1>': 'sub_p75'})
)
submarket_stats['sub_iqr'] = submarket_stats['sub_p75'] - submarket_stats['sub_p25']

df = df.merge(submarket_stats, left_on='submarket', right_index=True, how='left')

# ── 4b. Site-level comparison metrics ─────────────────────────────────
df['rent_z_score'] = (df['current_monthly_rent'] - df['sub_mean']) / df['sub_std']

df['rent_percentile_rank'] = (
    df.groupby('submarket')['current_monthly_rent']
    .rank(pct=True) * 100
)

df['deviation_from_median_pct'] = (
    (df['current_monthly_rent'] - df['sub_median']) / df['sub_median'] * 100
)

print('Submarket-level statistics:')
display(submarket_stats.sort_values('sub_median', ascending=False).round(0).head(10))

In [ ]:
# ── 4c. Ring-level aggregation for the demo search ring ──────────────
ring_df_in = ring_df_in.merge(
    submarket_stats[['sub_median', 'sub_std']],
    left_on='submarket', right_index=True, how='left'
)

ring_stats = (
    ring_df_in.groupby('ring_band')['current_monthly_rent']
    .agg(['count', 'median', 'mean', 'std',
          lambda x: x.quantile(0.25),
          lambda x: x.quantile(0.75)])
    .rename(columns={'count': 'sites', 'median': 'ring_median',
                     'mean': 'ring_mean', 'std': 'ring_std',
                     '<lambda_0>': 'ring_p25', '<lambda_1>': 'ring_p75'})
    .reindex(band_order)
)

ring_stats['ring_iqr'] = ring_stats['ring_p75'] - ring_stats['ring_p25']

print(f'Ring-level rent statistics (center = {DEMO_SUBMARKET}):')
display(ring_stats.round(0))

In [ ]:
# ── 4d. Rent distribution histogram within search ring ────────────────
fig_hist = px.histogram(
    ring_df_in, x='current_monthly_rent', color='ring_band',
    color_discrete_map=color_map,
    category_orders={'ring_band': band_order},
    nbins=40, barmode='overlay', opacity=0.65,
    title='Rent Distribution Within Search Rings',
    labels={'current_monthly_rent': 'Monthly Rent ($)'},
    height=400,
)
ring_median = ring_df_in['current_monthly_rent'].median()
fig_hist.add_vline(x=ring_median, line_dash='dash', line_color='black',
                   annotation_text=f'Ring Median: ${ring_median:,.0f}')
fig_hist.show()

In [ ]:
# ── 4e. Ranked deviation table (top overpaying sites in ring) ────────
ring_df_in['deviation_from_ring_median_pct'] = (
    (ring_df_in['current_monthly_rent'] - ring_median) / ring_median * 100
)

ranked_cols = ['site_id', 'submarket', 'ring_band', 'distance_mi',
               'current_monthly_rent', 'property_type', 'escalation_type',
               'lease_age_years', 'deviation_from_ring_median_pct']

ranked = (
    ring_df_in[ranked_cols]
    .sort_values('deviation_from_ring_median_pct', ascending=False)
    .head(20)
)

print(f'Top 20 sites by deviation from ring median (${ring_median:,.0f}/mo):')
display(ranked.round(1))

---
## Step 5 -- Multi-Factor Comparison Panels

Six factor groups, each showing how the selected ring's sites compare across
key dimensions.

In [ ]:
# ── Helper: normalise columns to 0-100 for radar charts ───────────────
def pctile_rank_cols(data, cols):
    """Return percentile-ranked (0-100) version of each column."""
    out = pd.DataFrame(index=data.index)
    for c in cols:
        out[c] = data[c].rank(pct=True) * 100
    return out

### 5a. Macro-Economic Factors

In [ ]:
macro_cols = ['unemployment_rate_local']

macro_summary = (
    ring_df_in.groupby('ring_band')[macro_cols]
    .median()
    .reindex(band_order)
)
macro_summary.loc['Submarket Overall'] = df[df['submarket'] == DEMO_SUBMARKET][macro_cols].median()

fig_macro = px.bar(
    macro_summary.reset_index().melt(id_vars='ring_band'),
    x='variable', y='value', color='ring_band',
    barmode='group',
    title='5a. Macro-Economic Factors by Ring Band',
    labels={'value': 'Median Value', 'variable': 'Factor'},
    height=400,
)
fig_macro.show()

### 5b. Micro-Economic Factors

In [ ]:
micro_cols = ['census_median_income', 'census_median_home_value',
              'census_population_density', 'underlying_land_value_psf',
              'hud_safmr', 'property_tax_rate']

ring_pctiles = pctile_rank_cols(ring_df_in, micro_cols)
ring_medians = ring_pctiles.median()
sub_pctiles = pctile_rank_cols(
    df[df['submarket'] == DEMO_SUBMARKET], micro_cols
).median()

fig_radar = go.Figure()
fig_radar.add_trace(go.Scatterpolar(
    r=ring_medians.values.tolist() + [ring_medians.values[0]],
    theta=micro_cols + [micro_cols[0]],
    fill='toself', name='Search Ring Median',
))
fig_radar.add_trace(go.Scatterpolar(
    r=sub_pctiles.values.tolist() + [sub_pctiles.values[0]],
    theta=micro_cols + [micro_cols[0]],
    fill='toself', name=f'{DEMO_SUBMARKET} Submarket',
))
fig_radar.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
    title='5b. Micro-Economic Profile (percentile rank)',
    height=500,
)
fig_radar.show()

### 5c. Zoning & Regulatory Factors

In [ ]:
zoning_dist = (
    ring_df_in.groupby(['ring_band', 'zoning_district_type'])
    .size().reset_index(name='count')
)

fig_zoning = px.bar(
    zoning_dist, x='ring_band', y='count', color='zoning_district_type',
    barmode='stack',
    category_orders={'ring_band': band_order},
    title='5c. Zoning Distribution by Ring Band',
    labels={'count': '# Sites', 'zoning_district_type': 'Zoning'},
    height=400,
)
fig_zoning.show()

zoning_rent = (
    ring_df_in.groupby('zoning_district_type')['current_monthly_rent']
    .agg(['median', 'count'])
    .sort_values('median', ascending=False)
)
print('Median rent by zoning type (within ring):')
display(zoning_rent.round(0))

### 5d. Technology & Network Factors

In [ ]:
tech_dist = (
    ring_df_in.groupby('technology_label')
    .agg(sites=('site_id', 'count'),
         median_rent=('current_monthly_rent', 'median'),
         median_tenants=('tenant_count', 'median'),
         median_antennas=('antenna_count', 'median'),
         pct_fiber=('has_fiber_connection', 'mean'),
         pct_critical=('coverage_critical', 'mean'))
    .sort_index()
)

print('Technology profile (within search ring):')
display(tech_dist.round(2))

fig_tech = make_subplots(rows=1, cols=2,
                         subplot_titles=['Technology Mix', 'Median Rent by Tech'],
                         specs=[[{'type': 'pie'}, {'type': 'bar'}]])

fig_tech.add_trace(
    go.Pie(labels=tech_dist.index, values=tech_dist['sites'], hole=0.4),
    row=1, col=1
)
fig_tech.add_trace(
    go.Bar(x=tech_dist.index, y=tech_dist['median_rent'],
           marker_color='steelblue'),
    row=1, col=2
)
fig_tech.update_layout(title='5d. Technology & Network Factors', height=400,
                       showlegend=False)
fig_tech.show()

### 5e. Competitive Landscape Factors

In [ ]:
fig_comp = px.scatter(
    ring_df_in, x='scarcity_index', y='current_monthly_rent',
    color='ring_band', color_discrete_map=color_map,
    size='fcc_tower_count_5km',
    hover_name='site_id',
    hover_data={'competitor_tower_distance_km': ':.1f',
                'landlord_concentration': True,
                'search_ring_alternative_count': True},
    title='5e. Scarcity Index vs Rent (bubble = tower density)',
    labels={'scarcity_index': 'Scarcity Index (higher = less scarce)',
            'current_monthly_rent': 'Monthly Rent ($)'},
    height=450,
)
fig_comp.show()

comp_summary = ring_df_in[[
    'fcc_tower_count_5km', 'fcc_tower_density_sqkm',
    'competitor_tower_distance_km', 'scarcity_index',
    'landlord_concentration', 'search_ring_alternative_count'
]].describe().round(2)

print('Competitive landscape summary (within ring):')
display(comp_summary)

### 5f. Contractual / Lease Factors

In [ ]:
# Escalation type distribution
esc_rent = (
    ring_df_in.groupby('escalation_type')['current_monthly_rent']
    .agg(['count', 'median', 'mean'])
    .sort_values('median', ascending=False)
)
print('Rent by escalation type (within ring):')
display(esc_rent.round(0))

fig_esc = px.box(
    ring_df_in, x='escalation_type', y='current_monthly_rent',
    color='escalation_type',
    title='5f-i. Rent Distribution by Escalation Type',
    labels={'current_monthly_rent': 'Monthly Rent ($)',
            'escalation_type': 'Escalation Type'},
    height=400,
)
fig_esc.show()

In [ ]:
# Lease age vs escalation drift
if 'escalation_drift_ratio' not in ring_df_in.columns:
    ring_df_in['escalation_drift_ratio'] = (
        ring_df_in['cumulative_escalation_factor']
        / (1 + CPI_AVG_ANNUAL) ** ring_df_in['lease_age_years']
    )

fig_drift = px.scatter(
    ring_df_in, x='lease_age_years', y='escalation_drift_ratio',
    color='escalation_type',
    size='current_monthly_rent',
    hover_name='site_id',
    hover_data={'current_monthly_rent': ':$,.0f',
                'cumulative_escalation_factor': ':.2f',
                'remaining_term_years': ':.1f'},
    title='5f-ii. Lease Age vs Escalation Drift Ratio',
    labels={'lease_age_years': 'Lease Age (years)',
            'escalation_drift_ratio': 'Drift Ratio (>1 = outpaced CPI)'},
    height=450,
)
fig_drift.add_hline(y=1.0, line_dash='dash', line_color='red',
                    annotation_text='CPI parity')
fig_drift.show()

In [ ]:
# Rent decomposition: base rent + escalation gap
if 'rent_vintage_gap' not in ring_df_in.columns:
    ring_df_in['rent_vintage_gap'] = (
        ring_df_in['current_monthly_rent'] - ring_df_in['de_escalated_base_rent']
    )

top_sites = (
    ring_df_in.nlargest(15, 'current_monthly_rent')
    .sort_values('current_monthly_rent', ascending=True)
)

fig_waterfall = go.Figure()
fig_waterfall.add_trace(go.Bar(
    y=top_sites['site_id'], x=top_sites['de_escalated_base_rent'],
    orientation='h', name='Base Rent (de-escalated)',
    marker_color='steelblue',
))
fig_waterfall.add_trace(go.Bar(
    y=top_sites['site_id'], x=top_sites['rent_vintage_gap'],
    orientation='h', name='Escalation Gap',
    marker_color='coral',
))
fig_waterfall.update_layout(
    barmode='stack',
    title='5f-iii. Rent Decomposition: Base Rent + Escalation Gap (top 15)',
    xaxis_title='Monthly Rent ($)',
    yaxis_title='Site ID',
    height=500,
)
fig_waterfall.show()

In [ ]:
# Lease-age vs escalation-type heatmap
ring_df_in['lease_age_bin'] = pd.cut(
    ring_df_in['lease_age_years'],
    bins=[0, 5, 10, 15, 20, 30],
    labels=['0-5yr', '5-10yr', '10-15yr', '15-20yr', '20+yr']
)

heatmap_data = (
    ring_df_in.groupby(['lease_age_bin', 'escalation_type'],
                       observed=False)['current_monthly_rent']
    .median()
    .unstack(fill_value=0)
)

fig_heat = px.imshow(
    heatmap_data.values,
    x=heatmap_data.columns.tolist(),
    y=heatmap_data.index.tolist(),
    color_continuous_scale='YlOrRd',
    text_auto='$,.0f',
    title='5f-iv. Median Rent Heatmap: Lease Age x Escalation Type',
    labels={'x': 'Escalation Type', 'y': 'Lease Age Bin', 'color': 'Median Rent ($)'},
    height=400,
)
fig_heat.show()

In [ ]:
# Remaining term timeline
fig_term = px.bar(
    ring_df_in.sort_values('remaining_term_years'),
    x='site_id', y='remaining_term_years',
    color=ring_df_in.sort_values('remaining_term_years')['remaining_term_years'].apply(
        lambda x: 'Near expiry (<3yr)' if x < 3 else ('Mid-term (3-10yr)' if x < 10 else 'Long-term (10+yr)')
    ),
    color_discrete_map={'Near expiry (<3yr)': '#d62728',
                        'Mid-term (3-10yr)': '#ff7f0e',
                        'Long-term (10+yr)': '#2ca02c'},
    title='5f-v. Remaining Lease Term (all ring sites)',
    labels={'remaining_term_years': 'Remaining Term (yrs)', 'color': 'Term Bucket'},
    height=400,
)
fig_term.update_xaxes(showticklabels=False, title='Sites (sorted by remaining term)')
fig_term.show()

near_expiry = (ring_df_in['remaining_term_years'] < 3).sum()
total_ring = len(ring_df_in)
print(f'Near-expiry sites (<3yr remaining): {near_expiry} / {total_ring} '
      f'({near_expiry/total_ring*100:.1f}%)')

---
## Step 6 -- Business KPIs

In [ ]:
def compute_portfolio_kpis(data, submarket_name='Portfolio'):
    """Compute 13 portfolio-level KPIs for a set of sites."""
    rent = data['current_monthly_rent']
    p25, p75 = rent.quantile(0.25), rent.quantile(0.75)

    has_4g = data['primary_technology'].isin([3])
    has_5g = data['primary_technology'].isin([4, 5])
    med_4g = rent[has_4g].median() if has_4g.any() else np.nan
    med_5g = rent[has_5g].median() if has_5g.any() else np.nan

    kpis = pd.Series({
        '1. Median Monthly Rent ($)': rent.median(),
        '2. Rent Dispersion IQR ($)': p75 - p25,
        '3. Overpayment Exposure Rate (%)': (rent > p75).mean() * 100,
        '4. Rent-to-Land-Value Ratio': (
            rent / data['underlying_land_value_psf'].clip(lower=0.01)
        ).median(),
        '5. Escalation Drift Index': data['escalation_drift_ratio'].median()
            if 'escalation_drift_ratio' in data.columns else np.nan,
        '6. Technology Premium (5G/4G)': (
            med_5g / med_4g if pd.notna(med_4g) and med_4g > 0 else np.nan
        ),
        '7. Competitive Pressure Score': (
            data['scarcity_index'].median() * data['landlord_concentration'].median()
        ),
        '8. Weighted Avg Lease Age (yr)': np.average(
            data['lease_age_years'], weights=rent
        ),
        '9. Near-Expiry Exposure (%)': (
            (data['remaining_term_years'] < 3).mean() * 100
        ),
        '10. Fixed-Escalator Dominance (%)': (
            data['escalation_type'].isin(['fixed_3pct', 'fixed_2pct']).mean() * 100
        ),
        '11. Amendment Coverage Rate (%)': (
            (data['amendment_count'] > 0).mean() * 100
        ),
        '12. Vintage Gap Ratio (%)': data['rent_vintage_gap_pct'].median()
            if 'rent_vintage_gap_pct' in data.columns else np.nan,
        '13. Portfolio Escalation Overshoot': (
            data['cumulative_escalation_factor'].median() - 1
        ),
    }, name=submarket_name)
    return kpis

In [ ]:
# ── 6a. Portfolio KPIs for demo submarket vs full portfolio ────────────
sub_data = df[df['submarket'] == DEMO_SUBMARKET]
kpi_sub = compute_portfolio_kpis(sub_data, DEMO_SUBMARKET)
kpi_all = compute_portfolio_kpis(df, 'All Submarkets')

kpi_compare = pd.DataFrame({'All Submarkets': kpi_all, DEMO_SUBMARKET: kpi_sub})

print(f'Portfolio-Level KPIs -- {DEMO_SUBMARKET} vs Overall:')
display(kpi_compare.round(2))

In [ ]:
# ── 6b. Site-level KPIs ──────────────────────────────────────────────
peer_median = df.groupby(['submarket', 'property_type'])['current_monthly_rent'].transform('median')
df['peer_comparison_index'] = df['current_monthly_rent'] / peer_median.clip(lower=1)

df['infrastructure_value_score'] = (
    df['tenant_count'] * df['antenna_count']
    / df['current_monthly_rent'].clip(lower=1)
)

df['lease_efficiency_ratio'] = (
    df['de_escalated_base_rent'] / df['hud_safmr'].clip(lower=1)
)

df['contractual_risk_score'] = (
    df['escalation_drift_ratio']
    * df['lease_age_years']
    / df['remaining_term_years'].clip(lower=0.5)
)

site_kpi_cols = ['site_id', 'submarket', 'current_monthly_rent',
                 'rent_z_score', 'rent_percentile_rank',
                 'peer_comparison_index', 'infrastructure_value_score',
                 'lease_efficiency_ratio', 'escalation_drift_ratio',
                 'rent_vintage_gap_pct', 'term_urgency_score',
                 'contractual_risk_score']

print(f'Site-level KPIs for {DEMO_SUBMARKET} (top 10 by contractual risk):')
display(
    df[df['submarket'] == DEMO_SUBMARKET][site_kpi_cols]
    .nlargest(10, 'contractual_risk_score')
    .round(2)
)

In [ ]:
# ── 6c. Trend / Comparative KPIs ─────────────────────────────────────

# Ring Rent Gradient
ring_gradient = (
    ring_df_in.groupby('ring_band')['current_monthly_rent']
    .median()
    .reindex(band_order)
)
print('C1. Ring Rent Gradient:')
for band, med in ring_gradient.items():
    print(f'   {band}: ${med:,.0f}')

# Property Type Premium
prop_premium = (
    ring_df_in.groupby('property_type')['current_monthly_rent']
    .median()
    .sort_values(ascending=False)
)
print(f'\nC2. Property Type Premium (ring median rent):')
for pt, med in prop_premium.items():
    print(f'   {pt}: ${med:,.0f}')

# Escalation Type Spread
esc_spread = (
    ring_df_in.groupby('escalation_type')['current_monthly_rent']
    .median()
    .sort_values(ascending=False)
)
print(f'\nC4. Escalation Type Spread (ring median rent):')
for et, med in esc_spread.items():
    print(f'   {et}: ${med:,.0f}')

# Lease Vintage Cohort
df['lease_age_bin'] = pd.cut(
    df['lease_age_years'],
    bins=[0, 5, 10, 15, 20, 30],
    labels=['0-5yr', '5-10yr', '10-15yr', '15-20yr', '20+yr']
)
vintage_cohort = (
    df[df['submarket'] == DEMO_SUBMARKET]
    .groupby('lease_age_bin', observed=False)['current_monthly_rent']
    .median()
)
print(f'\nC5. Lease Vintage Cohort ({DEMO_SUBMARKET} median rent):')
for cohort, med in vintage_cohort.items():
    print(f'   {cohort}: ${med:,.0f}')

---
## Step 7 -- Anomaly Detection (Statistical, No ML)

Five independent statistical methods. A site's `anomaly_severity` is the
count of flags triggered (0-5). Tier:
- **CRITICAL** >= 3 flags
- **WARNING** >= 2 flags
- **WATCH** = 1 flag
- **NORMAL** = 0 flags

In [ ]:
# ── 7a. IQR Fence Method (Tukey's Fences) ────────────────────────────
def compute_iqr_flag(group):
    rent = group['current_monthly_rent']
    q1, q3 = rent.quantile(0.25), rent.quantile(0.75)
    iqr = q3 - q1
    return ((rent < q1 - 1.5 * iqr) | (rent > q3 + 1.5 * iqr)).astype(int)

df['iqr_flag'] = df.groupby('submarket', group_keys=False).apply(
    lambda g: compute_iqr_flag(g)
).values

print(f'7a. IQR flags: {df["iqr_flag"].sum()} sites '
      f'({df["iqr_flag"].mean()*100:.1f}%)')

In [ ]:
# ── 7b. Z-Score Method ────────────────────────────────────────────────
df['zscore_flag'] = (df['rent_z_score'].abs() > 2.0).astype(int)

print(f'7b. Z-score flags: {df["zscore_flag"].sum()} sites '
      f'({df["zscore_flag"].mean()*100:.1f}%)')

In [ ]:
# ── 7c. MAD Method (Robust Non-Parametric) ───────────────────────────
def compute_mad_z(group):
    rent = group['current_monthly_rent']
    med = rent.median()
    mad = (rent - med).abs().median()
    mad_scaled = mad * 1.4826
    return (rent - med).abs() / max(mad_scaled, 1e-6)

df['mad_z'] = df.groupby('submarket', group_keys=False).apply(
    lambda g: compute_mad_z(g)
).values

df['mad_flag'] = (df['mad_z'] > 3.0).astype(int)

print(f'7c. MAD flags: {df["mad_flag"].sum()} sites '
      f'({df["mad_flag"].mean()*100:.1f}%)')

In [ ]:
# ── 7d. Multi-Factor Anomaly (Rule-Based) ────────────────────────────
pctile_cols = ['underlying_land_value_psf', 'census_median_income',
               'census_population_density', 'census_median_home_value',
               'hud_safmr']

for col in pctile_cols + ['current_monthly_rent', 'cumulative_escalation_factor']:
    df[f'{col}_pctile'] = (
        df.groupby('submarket')[col].rank(pct=True) * 100
    )

df['market_score'] = df[[f'{c}_pctile' for c in pctile_cols]].mean(axis=1)

df['multi_factor_flag'] = (
    (df['current_monthly_rent_pctile'] >= 75) &
    (df['market_score'] <= 40) &
    (df['cumulative_escalation_factor_pctile'] >= 60)
).astype(int)

print(f'7d-i. Multi-factor flags: {df["multi_factor_flag"].sum()} sites '
      f'({df["multi_factor_flag"].mean()*100:.1f}%)')

In [ ]:
# ── 7d-ii. Contractual Anomaly (sub-variant) ─────────────────────────
df['contractual_anomaly_flag'] = (
    (df['current_monthly_rent_pctile'] >= 75) &
    (df['escalation_type'].isin(['fixed_3pct', 'fixed_2pct'])) &
    (df['lease_age_years'] >= 15) &
    (df['amendment_count'] == 0)
).astype(int)

print(f'7d-ii. Contractual anomaly flags: {df["contractual_anomaly_flag"].sum()} sites '
      f'({df["contractual_anomaly_flag"].mean()*100:.1f}%)')

In [ ]:
# ── 7e. Anomaly Summary Table ────────────────────────────────────────
flag_cols = ['iqr_flag', 'zscore_flag', 'mad_flag',
             'multi_factor_flag', 'contractual_anomaly_flag']

df['anomaly_severity'] = df[flag_cols].sum(axis=1)

df['anomaly_tier'] = pd.cut(
    df['anomaly_severity'],
    bins=[-1, 0, 1, 2, 5],
    labels=['NORMAL', 'WATCH', 'WARNING', 'CRITICAL']
)

tier_counts = df['anomaly_tier'].value_counts().reindex(
    ['CRITICAL', 'WARNING', 'WATCH', 'NORMAL']
)

print('Anomaly Tier Distribution (all sites):')
for tier, cnt in tier_counts.items():
    print(f'  {tier:10s}: {cnt:>5} sites ({cnt/len(df)*100:.1f}%)')

In [ ]:
# Full anomaly summary table
anomaly_cols = [
    'site_id', 'submarket', 'current_monthly_rent', 'sub_median',
    'deviation_from_median_pct', 'rent_z_score', 'mad_z',
    'iqr_flag', 'zscore_flag', 'mad_flag',
    'multi_factor_flag', 'contractual_anomaly_flag',
    'anomaly_severity', 'anomaly_tier',
    'escalation_type', 'lease_age_years', 'remaining_term_years',
    'cumulative_escalation_factor', 'amendment_count',
    'property_type', 'market_score',
]

anomaly_table = (
    df[df['anomaly_severity'] >= 1][anomaly_cols]
    .sort_values(['anomaly_severity', 'deviation_from_median_pct'],
                 ascending=[False, False])
)

print(f'Anomaly Summary Table: {len(anomaly_table)} flagged sites '
      f'out of {len(df)} total')
display(anomaly_table.head(30).round(2))

In [ ]:
# ── Anomaly tier breakdown by submarket ──────────────────────────────
tier_by_sub = (
    df.groupby(['submarket', 'anomaly_tier'], observed=False)
    .size().unstack(fill_value=0)
    .reindex(columns=['CRITICAL', 'WARNING', 'WATCH', 'NORMAL'])
)
tier_by_sub['total_flagged'] = tier_by_sub[['CRITICAL', 'WARNING', 'WATCH']].sum(axis=1)
tier_by_sub = tier_by_sub.sort_values('total_flagged', ascending=False)

print('Anomaly breakdown by submarket (sorted by total flagged):')
display(tier_by_sub.head(15))

In [ ]:
# ── Anomaly visualisation: severity map ──────────────────────────────
flagged = df[df['anomaly_severity'] >= 1].copy()

tier_colors = {'CRITICAL': '#d62728', 'WARNING': '#ff7f0e',
               'WATCH': '#f0e442'}

fig_anomaly_map = px.scatter_mapbox(
    flagged, lat='lat', lon='lon',
    color='anomaly_tier',
    color_discrete_map=tier_colors,
    category_orders={'anomaly_tier': ['CRITICAL', 'WARNING', 'WATCH']},
    size='anomaly_severity',
    hover_name='site_id',
    hover_data={'current_monthly_rent': ':$,.0f',
                'submarket': True,
                'anomaly_severity': True,
                'deviation_from_median_pct': ':.1f'},
    title='Anomaly Map -- Flagged Sites Across All Submarkets',
    mapbox_style='carto-positron',
    zoom=3, center={'lat': 38, 'lon': -96},
    height=600,
)
fig_anomaly_map.update_layout(margin=dict(l=0, r=0, t=40, b=0))
fig_anomaly_map.show()

---
## Step 8 -- Executive Summary

In [ ]:
# ── 8a. Cross-submarket KPI comparison ───────────────────────────────
all_kpis = []
for sub in submarket_order:
    sub_data = df[df['submarket'] == sub]
    if len(sub_data) >= 10:
        all_kpis.append(compute_portfolio_kpis(sub_data, sub))

kpi_matrix = pd.DataFrame(all_kpis)
kpi_matrix.index.name = 'Submarket'

print('Portfolio KPIs by Submarket (sorted by Escalation Drift Index):')
display(
    kpi_matrix
    .sort_values('5. Escalation Drift Index', ascending=False)
    .round(2)
)

In [ ]:
# ── 8b. Renegotiation priority list (top 25 across all submarkets) ──
renego_cols = [
    'site_id', 'submarket', 'current_monthly_rent', 'sub_median',
    'deviation_from_median_pct', 'escalation_type', 'lease_age_years',
    'remaining_term_years', 'cumulative_escalation_factor',
    'escalation_drift_ratio', 'rent_vintage_gap_pct',
    'contractual_risk_score', 'anomaly_tier',
]

renego_list = (
    df[df['anomaly_severity'] >= 2][renego_cols]
    .sort_values('contractual_risk_score', ascending=False)
    .head(25)
)

print(f'Top 25 Renegotiation Priority Sites (severity >= WARNING):')
display(renego_list.round(2))

In [ ]:
# ── 8c. Summary statistics ───────────────────────────────────────────
critical = df[df['anomaly_tier'] == 'CRITICAL']
warning = df[df['anomaly_tier'] == 'WARNING']

total_monthly_rent = df['current_monthly_rent'].sum()
critical_monthly = critical['current_monthly_rent'].sum()
warning_monthly = warning['current_monthly_rent'].sum()

critical_overshoot = (critical['current_monthly_rent'] - critical['sub_median']).clip(lower=0).sum()
warning_overshoot = (warning['current_monthly_rent'] - warning['sub_median']).clip(lower=0).sum()

print('=' * 65)
print('EXECUTIVE SUMMARY')
print('=' * 65)
print(f'Total sites analysed            : {len(df):>8,}')
print(f'Total monthly rent              : ${total_monthly_rent:>12,.0f}')
print(f'')
print(f'CRITICAL anomalies              : {len(critical):>8,} sites')
print(f'  Monthly rent at risk          : ${critical_monthly:>12,.0f}')
print(f'  Overshoot vs submarket median : ${critical_overshoot:>12,.0f}/mo')
print(f'')
print(f'WARNING anomalies               : {len(warning):>8,} sites')
print(f'  Monthly rent at risk          : ${warning_monthly:>12,.0f}')
print(f'  Overshoot vs submarket median : ${warning_overshoot:>12,.0f}/mo')
print(f'')
print(f'Potential monthly savings target : ${critical_overshoot + warning_overshoot:>12,.0f}')
print(f'Annualised savings potential     : ${(critical_overshoot + warning_overshoot) * 12:>12,.0f}')
print(f'')
print(f'Fixed-escalator portfolio share  : {(df["escalation_type"].isin(["fixed_3pct", "fixed_2pct"]).mean()*100):.1f}%')
print(f'Median escalation drift index   : {df["escalation_drift_ratio"].median():.3f}')
print(f'Near-expiry sites (<3yr)        : {(df["remaining_term_years"] < 3).sum():>8,}')
print(f'Never-amended sites             : {(df["amendment_count"] == 0).sum():>8,} ({(df["amendment_count"] == 0).mean()*100:.1f}%)')
print('=' * 65)